# Data Augmentation
- EDA(Easy Data Augmentation)기법 사용
- https://arxiv.org/pdf/1901.11196.pdf
- 아래 4가지 방법중 하나를 선택하여 증강

> 1. SR(Synonym Replacement) : 불용어가 아닌 n개의 단어를 랜덤하게 뽑고 임의로 선택한 동의어와 바꾼다.
2. RI(Random Insertion) : 문장 안에 불용어가 아닌 임의의 단어를 선택해 해당 단어의 임의의 동의어를 랜덤한 위치에 삽입한다.(n번 반복)
3. RS(Random Swap) : 문장에서 임의의 두 단어를 선택하고 자리를 바꾼다.(n번 반복)
4. RD(Random Delection) : 문장에서 임의의 단어를 p의 확률로 삭제

- EDA를 한국어로 쓸 수 있도록 wordnet부분만 교체한 KorEDA 사용
(https://github.com/catSirup/KorEDA)

In [94]:
import pandas as pd
import csv
import numpy as np

In [1]:
!git clone https://github.com/catSirup/KorEDA.git

Cloning into 'KorEDA'...


In [16]:
cd KorEDA

C:\Users\Yong\코드스테이츠 AI\프리온보딩\KorEDA


In [17]:
from eda import *

In [18]:
cd ..

C:\Users\Yong\코드스테이츠 AI\프리온보딩


In [66]:
# 원본 데이터
train_ = pd.read_json('./klue-sts-v1.1/klue-sts-v1.1_train.json') # 11668x6
test_ = pd.read_json('./klue-sts-v1.1/klue-sts-v1.1_dev.json')    # 519x6

In [67]:
test_.head()

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_dev_00000,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
1,klue-sts-v1_dev_00001,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,"{'label': 1.4, 'real-label': 1.428571428571429...","{'agreement': '0:4:3:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_dev_00002,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,"{'label': 1.3, 'real-label': 1.285714285714286...","{'agreement': '0:5:2:0:0:0', 'annotators': ['0..."
3,klue-sts-v1_dev_00003,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.","{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['1..."
4,klue-sts-v1_dev_00004,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,"{'label': 2.5, 'real-label': 2.5, 'binary-labe...","{'agreement': '1:0:1:3:1:0', 'annotators': ['0..."


In [68]:
train_['sentence1'][0]

'숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.'

In [69]:
EDA(train_['sentence1'][0], num_aug=5)    # 예시

['반지하 위치는 찾기 쉽고 일반적인 한국의 숙소 숙소입니다.',
 '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙소 위치는 찾기 숙 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.']

### 한계점
- wordnet만을 단순히 바꿔서 결과를 내기때문에 의미가 변형되는 경우가 생김
- 특히 SR,RI에서 많이 발생 예) 잘못한 건 아니다 -> 잘못한 총 아니다
- 안전하게 데이터 증강하기 위하여 **RD, RS**만 사용하여 증강

In [70]:
# EDA를 사용한 후에도 중복된 문장 제거필요

def remove_duplicate(x):
    return list(dict.fromkeys(x))

In [71]:
augment = []
new_sentence = test_['sentence1']

for x in new_sentence:
    aug_list = remove_duplicate(EDA(x, num_aug=5))
    augment.append(aug_list)

In [72]:
augment[0]

['호스트분들이 무엇보다도 너무 친절하셨습니다.',
 '친절하셨습니다. 호스트분들이 너무 무엇보다도',
 '무엇보다도 호스트분들이 너무 친절하셨습니다.']

In [73]:
z = len(test_)
for x in range(len(augment)):
    for y in range(len(augment[x])):
        test_ = test_.append(test_.loc[x], ignore_index=True)
        z += 1
        test_.loc[z, 'sentence1'] = augment[x][y]

In [74]:
len(test_)   # 519 -> 2436

2436

In [79]:
test_.loc[0]

guid                                       klue-sts-v1_dev_00000
source                                                airbnb-rtt
sentence1                               무엇보다도 호스트분들이 너무 친절하셨습니다.
sentence2                                무엇보다도, 호스트들은 매우 친절했습니다.
labels         {'label': 4.9, 'real-label': 4.857142857142857...
annotations    {'agreement': '0:0:0:0:1:6', 'annotators': ['1...
Name: 0, dtype: object

In [81]:
test_.loc[521]

guid                                       klue-sts-v1_dev_00000
source                                                airbnb-rtt
sentence1                               친절하셨습니다. 호스트분들이 너무 무엇보다도
sentence2                                무엇보다도, 호스트들은 매우 친절했습니다.
labels         {'label': 4.9, 'real-label': 4.857142857142857...
annotations    {'agreement': '0:0:0:0:1:6', 'annotators': ['1...
Name: 521, dtype: object

In [82]:
# 중복 확인
test_.duplicated(['sentence1', 'sentence2']).sum()

520

In [87]:
# 결측치 확인
test_.isnull().sum().sum()

5

In [89]:
# 중복, 결측치 행 제거
test_ = test_.drop_duplicates(['sentence1', 'sentence2'], keep='first', ignore_index=True)
test_.dropna(axis=0, inplace=True)

C:\Users\Yong\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [91]:
test_.isnull().sum()

guid           0
source         0
sentence1      0
sentence2      0
labels         0
annotations    0
dtype: int64

In [92]:
test_.duplicated(['sentence1', 'sentence2']).sum()

0

In [93]:
test_

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_dev_00000,airbnb-rtt,무엇보다도 호스트분들이 너무 친절하셨습니다.,"무엇보다도, 호스트들은 매우 친절했습니다.","{'label': 4.9, 'real-label': 4.857142857142857...","{'agreement': '0:0:0:0:1:6', 'annotators': ['1..."
1,klue-sts-v1_dev_00001,airbnb-sampled,주요 관광지 모두 걸어서 이동가능합니다.,위치는 피렌체 중심가까지 걸어서 이동 가능합니다.,"{'label': 1.4, 'real-label': 1.428571428571429...","{'agreement': '0:4:3:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_dev_00002,policy-sampled,학생들의 균형 있는 영어능력을 향상시킬 수 있는 학교 수업을 유도하기 위해 2018...,영어 영역의 경우 학생들이 한글 해석본을 암기하는 문제를 해소하기 위해 2016학년...,"{'label': 1.3, 'real-label': 1.285714285714286...","{'agreement': '0:5:2:0:0:0', 'annotators': ['0..."
3,klue-sts-v1_dev_00003,airbnb-rtt,"다만, 도로와 인접해서 거리의 소음이 들려요.","하지만, 길과 가깝기 때문에 거리의 소음을 들을 수 있습니다.","{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['1..."
4,klue-sts-v1_dev_00004,paraKQC-para,형이 다시 캐나다 들어가야 하니 가족모임 일정은 바꾸지 마세요.,가족 모임 일정은 바꾸지 말도록 하십시오.,"{'label': 2.5, 'real-label': 2.5, 'binary-labe...","{'agreement': '1:0:1:3:1:0', 'annotators': ['0..."
...,...,...,...,...,...,...
1911,klue-sts-v1_dev_00516,airbnb-sampled,게스트에 배려가 대한 묻어나는 시설들이었습니다.,우선 공간에 대한 센스가 돋보이는 곳이었습니다.,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
1912,klue-sts-v1_dev_00517,policy-sampled,밤하늘을 펼쳐진다. ‘비대면 드론쇼’도 배경으로,‘비대면 실감형 문화공연 플랫폼’ 개념도.,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '5:0:1:0:0:0', 'annotators': ['0..."
1913,klue-sts-v1_dev_00517,policy-sampled,드론쇼’도 배경으로 ‘비대면 밤하늘을 펼쳐진다.,‘비대면 실감형 문화공연 플랫폼’ 개념도.,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '5:0:1:0:0:0', 'annotators': ['0..."
1914,klue-sts-v1_dev_00518,airbnb-sampled,엘리베이터는 포르투갈의 비앤비와 같이 여느 없습니다.,포르투의 거의 모든 숙박 시설은 엘리베이터는 없습니다.,"{'label': 2.9, 'real-label': 2.857142857142857...","{'agreement': '0:0:2:4:1:0', 'annotators': ['0..."


In [95]:
# augmentation 함수화
def remove_duplicate(x):
    return list(dict.fromkeys(x))

def augmentation(data, num_aug):     # 데이터, 문장 당 증강할 수
    augment = []
    z = len(data)
    sents = data['sentence1']

    for i in sents:
        aug_list = remove_duplicate(EDA(i, num_aug=num_aug))  # EDA(rd,rs)한 문장 리스트
        augment.append(aug_list)

    for x in range(len(augment)):
        for y in range(len(augment[x])):
            data = data.append(data.loc[x], ignore_index=True)
            z += 1
            data.loc[z, 'sentence1'] = augment[x][y]
            
    data = data.drop_duplicates(['sentence1', 'sentence2'], keep='first', ignore_index=True)
    data.dropna(axis=0, inplace=True)  # 중복 및 결측치 행 제거
    
    return data

In [98]:
train_aug = augmentation(train_, 5)
train_aug     # 11668 -> 44370

,guid,source,sentence1,sentence2,labels,annotations
0,klue-sts-v1_train_00000,airbnb-rtt,숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.,숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.,"{'label': 3.7, 'real-label': 3.714285714285714...","{'agreement': '0:0:0:2:5:0', 'annotators': ['0..."
1,klue-sts-v1_train_00001,policy-sampled,위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.,시민들 스스로 자발적인 예방 노력을 한 것은 아산 뿐만이 아니었다.,"{'label': 0.0, 'real-label': 0.0, 'binary-labe...","{'agreement': '5:0:0:0:0:0', 'annotators': ['1..."
2,klue-sts-v1_train_00002,paraKQC-sampled,회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.,사람들이 주로 네이버 메일을 쓰는 이유를 알려줘,"{'label': 0.30000000000000004, 'real-label': 0...","{'agreement': '4:2:0:0:0:0', 'annotators': ['1..."
3,klue-sts-v1_train_00003,policy-sampled,"긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업...","고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.","{'label': 0.6000000000000001, 'real-label': 0....","{'agreement': '4:2:1:0:0:0', 'annotators': ['1..."
4,klue-sts-v1_train_00004,airbnb-rtt,"호스트의 답장이 늦으나, 개선될 것으로 보입니다.",호스트 응답이 늦었지만 개선될 것으로 보입니다.,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."
...,...,...,...,...,...,...
44366,klue-sts-v1_train_11666,airbnb-rtt,개 1마리 개 고양이 3마리 너무 귀여워요!,개 한 마리와 고양이 세 마리는 정말 귀여워요!,"{'label': 4.7, 'real-label': 4.714285714285714...","{'agreement': '0:0:0:0:2:5', 'annotators': ['1..."
44367,klue-sts-v1_train_11667,paraKQC-para,학회 홍보 메일은 회신 메일을 학 보내지마,학회 홍보 메일은 회신 하지마,"{'label': 3.3, 'real-label': 3.333333333333333...","{'agreement': '0:0:1:2:3:0', 'annotators': ['1..."
44368,klue-sts-v1_train_11667,paraKQC-para,학 홍보 메일은 회신 메일을 보내지마,학회 홍보 메일은 회신 하지마,"{'label': 3.3, 'real-label': 3.333333333333333...","{'agreement': '0:0:1:2:3:0', 'annotators': ['1..."
44369,klue-sts-v1_train_11667,paraKQC-para,홍보 회신 메일을 보내지마,학회 홍보 메일은 회신 하지마,"{'label': 3.3, 'real-label': 3.333333333333333...","{'agreement': '0:0:1:2:3:0', 'annotators': ['1..."


In [104]:
train_aug['sentence1'].isnull().sum()

0